## 2023 해군 AI 전문인력 양성과정 Day 01: Tutorial 3 (HuggingFace Transformers)
### TA: Jihyoung Jang (jihyoung@unist.ac.kr)

In [ ]:
!pip install transformers==4.28.0
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.3 MB/s eta 0:00:00


In [ ]:
!pip install datasets
from datasets import Dataset, load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00


In [ ]:
!pip install evaluate
import evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.4 MB/s eta 0:00:00


In [ ]:
# snli = load_dataset("snli")
train, valid, test = load_dataset("snli", split=["train[:10%]", "validation[:10%]", "test[:10%]"])

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 55015
})

In [ ]:
train = train.filter(lambda example: example["label"] in [0, 1, 2])
valid = valid.filter(lambda example: example["label"] in [0, 1, 2])
test = test.filter(lambda example: example["label"] in [0, 1, 2])

Filter:   0%|          | 0/55015 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 54958
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
train[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1}

In [ ]:
premise = "A person on a horse jumps over a broken down airplane."

In [ ]:
encoding_premise = tokenizer(premise)
print(encoding_premise)

{'input_ids': [101, 1037, 2711, 2006, 1037, 3586, 14523, 2058, 1037, 3714, 2091, 13297, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
tokenizer.convert_ids_to_tokens(encoding_premise["input_ids"])

['[CLS]',
 'a',
 'person',
 'on',
 'a',
 'horse',
 'jumps',
 'over',
 'a',
 'broken',
 'down',
 'airplane',
 '.',
 '[SEP]']

In [ ]:
hypothesis = "A person is training his horse for a competition."

In [ ]:
encoding_hypothesis = tokenizer(hypothesis)
print(encoding_hypothesis)

{'input_ids': [101, 1037, 2711, 2003, 2731, 2010, 3586, 2005, 1037, 2971, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
tokenizer.convert_ids_to_tokens(encoding_hypothesis["input_ids"])

['[CLS]',
 'a',
 'person',
 'is',
 'training',
 'his',
 'horse',
 'for',
 'a',
 'competition',
 '.',
 '[SEP]']

In [ ]:
tokenizer(train[0]["premise"], train[0]["hypothesis"], truncation=True)

{'input_ids': [101, 1037, 2711, 2006, 1037, 3586, 14523, 2058, 1037, 3714, 2091, 13297, 1012, 102, 1037, 2711, 2003, 2731, 2010, 3586, 2005, 1037, 2971, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
train[100]

{'premise': 'A woman is walking across the street eating a banana, while a man is following with his briefcase.',
 'hypothesis': 'the woman is a seductress',
 'label': 1}

In [ ]:
tokenizer(train[100]["premise"], train[1]["hypothesis"], truncation=True)

{'input_ids': [101, 1037, 2450, 2003, 3788, 2408, 1996, 2395, 5983, 1037, 15212, 1010, 2096, 1037, 2158, 2003, 2206, 2007, 2010, 21793, 1012, 102, 1037, 2711, 2003, 2012, 1037, 15736, 1010, 13063, 2019, 18168, 12260, 4674, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

In [ ]:
tokenized_train = train.map(tokenize_function, batched=True, num_proc=8)
tokenized_valid = valid.map(tokenize_function, batched=True, num_proc=8)
tokenized_test = test.map(tokenize_function, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/54958 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/980 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/988 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 54958
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
sample_batch = tokenized_train[:16]
sample_batch = {k: v for k, v in sample_batch.items() if k not in ["premise", "hypothesis"]}
[len(x) for x in sample_batch["input_ids"]]

[25, 28, 24, 15, 13, 13, 26, 26, 25, 47, 37, 39, 18, 16, 17, 27]

In [ ]:
sample_batch = data_collator(sample_batch)
{k: v.shape for k, v in sample_batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([16, 47]),
 'token_type_ids': torch.Size([16, 47]),
 'attention_mask': torch.Size([16, 47]),
 'labels': torch.Size([16])}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 15.2 MB/s eta 0:00:00


In [ ]:
training_args = TrainingArguments(
    output_dir="./snli",
    learning_rate=5e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
)

In [ ]:
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
  )

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.363404,0.873469
2,0.524400,0.353535,0.861224
3,0.294000,0.410123,0.863265
4,0.180000,0.451615,0.861224


TrainOutput(global_step=1720, training_loss=0.3082492340442746, metrics={'train_runtime': 655.3255, 'train_samples_per_second': 335.455, 'train_steps_per_second': 2.625, 'total_flos': 6563515289882796.0, 'train_loss': 0.3082492340442746, 'epoch': 4.0})

In [ ]:
metric = trainer.evaluate(eval_dataset=tokenized_test)
print(metric)

{'eval_loss': 0.3609548807144165, 'eval_accuracy': 0.8593117408906883, 'eval_runtime': 0.8837, 'eval_samples_per_second': 1117.982, 'eval_steps_per_second': 9.052, 'epoch': 4.0}


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
inference = tokenizer("I love the principles of deep learning", "I hate the principles of deep learning", return_tensors="pt").to(device)

In [ ]:
# 0: entailment, 1: neutral, 2: contradiction

outputs = model(**inference)
print(outputs.logits.argmax(dim=-1)[0].tolist())

2
